# OptionsLab: End-to-End Quantitative Finance Demo

This notebook demonstrates the full capabilities of OptionsLab:
1. **Market Data Fetching** - Real option chain data from Yahoo Finance
2. **Implied Volatility Surface** - Computing IV from market prices
3. **Option Pricing Models** - Black-Scholes, Monte Carlo, ML Surrogate
4. **Exotic Options** - Asian, Barrier, American options
5. **Risk Analysis** - VaR, Greeks, Sensitivity Analysis

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

# OptionsLab imports
from src.pricing_models import (
    black_scholes,
    MonteCarloPricer,
    MonteCarloMLSurrogate,
    implied_volatility,
    AsianOption,
    BarrierOption,
    AmericanOption,
)
from src.utils.market_data import YahooFinanceFetcher, OptionChainParser
from src.risk_analysis import var, expected_shortfall

print("OptionsLab loaded successfully!")

## 1. Fetch Market Data

In [ ]:
# Fetch real option chain data
ticker = "AAPL"
fetcher = YahooFinanceFetcher()

# Get stock data
stock = fetcher.get_stock_data(ticker, period="6mo")
print(f"Current {ticker} price: ${stock.current_price:.2f}")

# Get option chain
option_chain = fetcher.get_option_chain(ticker)
print(f"Available expirations: {len(option_chain.expiration_dates)}")
print(f"Total calls: {len(option_chain.calls)}, Total puts: {len(option_chain.puts)}")

In [ ]:
# Parse and filter for liquid options
parsed = OptionChainParser.parse_yahoo_chain(option_chain)
liquid = OptionChainParser.filter_liquid_options(
    parsed, min_volume=50, min_open_interest=500
)
print(f"Liquid options: {len(liquid)}")
liquid.head(10)

## 2. Compute Implied Volatility Surface

In [ ]:
# Compute IV for each option
spot = stock.current_price
r = 0.05  # Risk-free rate

ivs = []
for _, row in liquid.iterrows():
    try:
        iv = implied_volatility(
            market_price=row["mid_price"],
            S=spot,
            K=row["strike"],
            T=row["T"],
            r=r,
            option_type=row["option_type"],
        )
        ivs.append(iv)
    except:
        ivs.append(np.nan)

liquid["computed_iv"] = ivs
print(f"Computed IVs for {liquid['computed_iv'].notna().sum()} options")

In [ ]:
# Plot IV smile for nearest expiration
calls = liquid[(liquid["option_type"] == "call") & (liquid["computed_iv"].notna())]
nearest_exp = calls["expiration"].min()
smile_data = calls[calls["expiration"] == nearest_exp].sort_values("strike")

plt.figure(figsize=(10, 5))
plt.plot(smile_data["moneyness"], smile_data["computed_iv"] * 100, "bo-", markersize=6)
plt.axvline(x=1.0, color="r", linestyle="--", alpha=0.5, label="ATM")
plt.xlabel("Moneyness (K/S)")
plt.ylabel("Implied Volatility (%)")
plt.title(f"{ticker} IV Smile - {nearest_exp}")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Compare Pricing Models

In [ ]:
# Test parameters
S, K, T, r, sigma = 100, 100, 1.0, 0.05, 0.2

# Black-Scholes (analytical)
bs_result = black_scholes(S, K, T, r, sigma, option_type="call")
print(f"Black-Scholes:     ${bs_result['price']:.4f}")
print(f"  Delta: {bs_result['delta']:.4f}, Gamma: {bs_result['gamma']:.4f}")

# Monte Carlo
mc = MonteCarloPricer(num_simulations=100000, seed=42)
mc_price = mc.price(S, K, T, r, sigma, option_type="call")
print(f"Monte Carlo:       ${mc_price:.4f}")

# ML Surrogate
ml = MonteCarloMLSurrogate(n_estimators=300, seed=42)
ml.fit(n_samples=10000, option_type="call", verbose=False)
ml_result = ml.predict_single(S, K, T, r, sigma, 0.0)
print(f"ML Surrogate:      ${ml_result['price']:.4f}")

In [ ]:
# Timing comparison
import time

# Time BS
start = time.perf_counter()
for _ in range(1000):
    black_scholes(S, K, T, r, sigma)
bs_time = (time.perf_counter() - start) / 1000 * 1000

# Time MC
start = time.perf_counter()
for _ in range(10):
    mc.price(S, K, T, r, sigma)
mc_time = (time.perf_counter() - start) / 10 * 1000

print(f"Black-Scholes: {bs_time:.3f} ms/call")
print(f"Monte Carlo:   {mc_time:.1f} ms/call")
print(f"Speedup:       {mc_time/bs_time:.0f}x")

## 4. Exotic Options

In [ ]:
# Asian Option (path-dependent)
asian = AsianOption(S=100, K=100, T=1.0, r=0.05, sigma=0.2, seed=42)
asian_arith = asian.price(n_paths=50000, avg_type="arithmetic", option_type="call")
asian_geom = asian.price_geometric_closed_form(option_type="call")

print("Asian Options:")
print(f"  Arithmetic avg (MC): ${asian_arith:.4f}")
print(f"  Geometric avg (CF):  ${asian_geom:.4f}")
print(f"  European (BS):       ${bs_result['price']:.4f}")
print(f"  Asian discount:      {(1 - asian_arith/bs_result['price'])*100:.1f}%")

In [ ]:
# Barrier Options
barrier_out = BarrierOption(
    S=100, K=100, T=1.0, r=0.05, sigma=0.2, barrier=120, seed=42
)
barrier_in = BarrierOption(S=100, K=100, T=1.0, r=0.05, sigma=0.2, barrier=120, seed=42)

up_out = barrier_out.price(n_paths=50000, barrier_type="up-and-out", option_type="call")
up_in = barrier_in.price(n_paths=50000, barrier_type="up-and-in", option_type="call")

print("Barrier Options (barrier=120):")
print(f"  Up-and-Out Call: ${up_out:.4f}")
print(f"  Up-and-In Call:  ${up_in:.4f}")
print(f"  Sum (≈European): ${up_out + up_in:.4f}")
print(f"  European:        ${bs_result['price']:.4f}")

In [ ]:
# American Put (early exercise premium)
from scipy.stats import norm


def bs_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


american = AmericanOption(S=100, K=100, T=1.0, r=0.05, sigma=0.2, seed=42)
american_price = american.price(n_paths=30000, option_type="put")
european_put = bs_put(100, 100, 1.0, 0.05, 0.2)

print("American vs European Put:")
print(f"  European Put:      ${european_put:.4f}")
print(f"  American Put:      ${american_price:.4f}")
print(
    f"  Early Ex. Premium: ${american_price - european_put:.4f} ({(american_price/european_put - 1)*100:.1f}%)"
)

## 5. Risk Analysis

In [ ]:
# Simulate portfolio returns
np.random.seed(42)
n_days = 252
returns = np.random.normal(0.0005, 0.02, n_days)  # Daily returns

# Calculate VaR and ES
portfolio_value = 1_000_000
var_95 = var.historical_var(returns, confidence=0.95) * portfolio_value
es_95 = expected_shortfall.historical_es(returns, confidence=0.95) * portfolio_value

print(f"Portfolio Value: ${portfolio_value:,.0f}")
print(f"VaR (95%):       ${var_95:,.0f}")
print(f"Expected Shortfall: ${es_95:,.0f}")

In [ ]:
# Greeks sensitivity
spots = np.linspace(80, 120, 21)
deltas = []
gammas = []
prices = []

for s in spots:
    result = black_scholes(s, K=100, T=1.0, r=0.05, sigma=0.2)
    deltas.append(result["delta"])
    gammas.append(result["gamma"])
    prices.append(result["price"])

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].plot(spots, prices, "b-", linewidth=2)
axes[0].set_xlabel("Spot Price")
axes[0].set_ylabel("Option Price")
axes[0].set_title("Price vs Spot")
axes[0].grid(True, alpha=0.3)

axes[1].plot(spots, deltas, "g-", linewidth=2)
axes[1].set_xlabel("Spot Price")
axes[1].set_ylabel("Delta")
axes[1].set_title("Delta vs Spot")
axes[1].axhline(y=0.5, color="r", linestyle="--", alpha=0.5)
axes[1].grid(True, alpha=0.3)

axes[2].plot(spots, gammas, "r-", linewidth=2)
axes[2].set_xlabel("Spot Price")
axes[2].set_ylabel("Gamma")
axes[2].set_title("Gamma vs Spot")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- ✅ Fetching real market data from Yahoo Finance
- ✅ Computing implied volatility surfaces
- ✅ Comparing Black-Scholes, Monte Carlo, and ML pricing
- ✅ Pricing exotic options (Asian, Barrier, American)
- ✅ Risk metrics (VaR, Expected Shortfall)
- ✅ Greeks sensitivity analysis